In [1]:
# Library Imports
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer

import numpy as np
np.set_printoptions(precision=4)

import matplotlib.pyplot as plt
plt.style.use('dark_background')

import os
from time import sleep

In [2]:
# Read the .stl files
package_dir = os.path.abspath(os.getcwd())
urdf_path = package_dir + '/model/fingeredu.urdf'

In [3]:
# Build the robot 
model, collision_model, visual_model = pin.buildModelsFromUrdf(urdf_path, package_dir)
data = model.createData()

In [4]:
# Display the visual of the robot
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()
q0 = pin.neutral(model)
viz.display(q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/


In [5]:
# Get the FrameID of the EOAT
EOAT_ID = model.getFrameId('finger_tip_link')

In [6]:
# Computing Forward Kinematics
def compute_ForwardKinematics(model, data, q):
    pin.framesForwardKinematics(model, data, q)
    pos = np.array(data.oMf[EOAT_ID].translation)
    return pos

In [7]:
# Compute the Jacobian
def compute_Jacobian(model, data, q):
    pin.computeJointJacobians(model, data, q)
    return pin.getFrameJacobian(model, data, EOAT_ID, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)[:3]

In [11]:
# Trajectory Planning for Point-to-Point(P2P) Motion
def P2P_Planner(model, data, joint_pos, goal_pos, interpolation='joint'):
    # Initiate Callers
    q = joint_pos
    y_target = goal_pos.copy()
    init_y = compute_ForwardKinematics(model, data, q)
    Error = goal_pos - init_y

    # Constants
    W = 1e-4 * np.eye(3)

    # HyperParams-> Needs to be tuned for large P2P Motion
    if interpolation=='joint':
        max_steps = 100
        smooth = 0.1
    if interpolation=='linear':
        max_steps = 1000
        smooth = 0.9

    for i in range(1, max_steps):
        y = compute_ForwardKinematics(model, data, q)
        J = compute_Jacobian(model, data, q) 

        # Compute Error
        Error = goal_pos - y

        if interpolation=='joint':
            pass
        if interpolation=='linear':
            y_target = init_y + (i/max_steps)*(goal_pos-y)

        # Compute Step Angle
        q += smooth * np.linalg.inv(J.T @ J + W) @ J.T @ (y_target - y)
        viz.display(q)
        sleep(2e-3)

    return q

In [14]:
# Point based Trajectory Planner
def Trajectory_Planner(model, data, joint_pos, Point_List):
    q = joint_pos
    # Invoke P2P_Planner for each Point
    for i, point in enumerate(Point_List):
        print (point)
        q = P2P_Planner(model, data, q, point)